<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

**Visualization of the probability density**

Plot the probability density in three dimensions for increasing
concentration parameter. The density is calculated by the ``pdf``
method.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import vonmises_fisher
from matplotlib.colors import Normalize
n_grid = 100
u = np.linspace(0, np.pi, n_grid)
v = np.linspace(0, 2 * np.pi, n_grid)
u_grid, v_grid = np.meshgrid(u, v)
vertices = np.stack([np.cos(v_grid) * np.sin(u_grid),
                     np.sin(v_grid) * np.sin(u_grid),
                     np.cos(u_grid)],
                    axis=2)
x = np.outer(np.cos(v), np.sin(u))
y = np.outer(np.sin(v), np.sin(u))
z = np.outer(np.ones_like(u), np.cos(u))
def plot_vmf_density(ax, x, y, z, vertices, mu, kappa):
    vmf = vonmises_fisher(mu, kappa)
    pdf_values = vmf.pdf(vertices)
    pdfnorm = Normalize(vmin=pdf_values.min(), vmax=pdf_values.max())
    ax.plot_surface(x, y, z, rstride=1, cstride=1,
                    facecolors=plt.cm.viridis(pdfnorm(pdf_values)),
                    linewidth=0)
    ax.set_aspect('equal')
    ax.view_init(azim=-130, elev=0)
    ax.axis('off')
    ax.set_title(rf"$\kappa={kappa}$")
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(9, 4),
                         subplot_kw={"projection": "3d"})
left, middle, right = axes
mu = np.array([-np.sqrt(0.5), -np.sqrt(0.5), 0])
plot_vmf_density(left, x, y, z, vertices, mu, 5)
plot_vmf_density(middle, x, y, z, vertices, mu, 20)
plot_vmf_density(right, x, y, z, vertices, mu, 100)
plt.subplots_adjust(top=1, bottom=0.0, left=0.0, right=1.0, wspace=0.)
plt.show()

As we increase the concentration parameter, the points are getting more
clustered together around the mean direction.

**Sampling**

Draw 5 samples from the distribution using the ``rvs`` method resulting
in a 5x3 array.


In [ ]:
rng = np.random.default_rng()
mu = np.array([0, 0, 1])
samples = vonmises_fisher(mu, 20).rvs(5, random_state=rng)
samples

array([[ 0.3884594 , -0.32482588,  0.86231516],
       [ 0.00611366, -0.09878289,  0.99509023],
       [-0.04154772, -0.01637135,  0.99900239],
       [-0.14613735,  0.12553507,  0.98126695],
       [-0.04429884, -0.23474054,  0.97104814]])

These samples are unit vectors on the sphere $S^2$. To verify,
let us calculate their euclidean norms:


In [ ]:
np.linalg.norm(samples, axis=1)

array([1., 1., 1., 1., 1.])

Plot 20 observations drawn from the von Mises-Fisher distribution for
increasing concentration parameter $\kappa$. The red dot highlights
the mean direction $\mu$.


In [ ]:
def plot_vmf_samples(ax, x, y, z, mu, kappa):
    vmf = vonmises_fisher(mu, kappa)
    samples = vmf.rvs(20)
    ax.plot_surface(x, y, z, rstride=1, cstride=1, linewidth=0,
                    alpha=0.2)
    ax.scatter(samples[:, 0], samples[:, 1], samples[:, 2], c='k', s=5)
    ax.scatter(mu[0], mu[1], mu[2], c='r', s=30)
    ax.set_aspect('equal')
    ax.view_init(azim=-130, elev=0)
    ax.axis('off')
    ax.set_title(rf"$\kappa={kappa}$")
mu = np.array([-np.sqrt(0.5), -np.sqrt(0.5), 0])
fig, axes = plt.subplots(nrows=1, ncols=3,
                         subplot_kw={"projection": "3d"},
                         figsize=(9, 4))
left, middle, right = axes
plot_vmf_samples(left, x, y, z, mu, 5)
plot_vmf_samples(middle, x, y, z, mu, 20)
plot_vmf_samples(right, x, y, z, mu, 100)
plt.subplots_adjust(top=1, bottom=0.0, left=0.0,
                    right=1.0, wspace=0.)
plt.show()

The plots show that with increasing concentration $\kappa$ the
resulting samples are centered more closely around the mean direction.

**Fitting the distribution parameters**

The distribution can be fitted to data using the ``fit`` method returning
the estimated parameters. As a toy example let's fit the distribution to
samples drawn from a known von Mises-Fisher distribution.


In [ ]:
mu, kappa = np.array([0, 0, 1]), 20
samples = vonmises_fisher(mu, kappa).rvs(1000, random_state=rng)
mu_fit, kappa_fit = vonmises_fisher.fit(samples)
mu_fit, kappa_fit

(array([0.01126519, 0.01044501, 0.99988199]), 19.306398751730995)

We see that the estimated parameters `mu_fit` and `kappa_fit` are
very close to the ground truth parameters.
